Generating plots that introduce the thesis. This notebook is for generating publication-ready plots, which will be automatically exported to the LaTeX directory. To follow the process of exploring the data, look at the other notebooks.

In [ ]:
# computation
import lumapi
import numpy as np
import warnings
import xarray as xr
import pandas as pd
# import xyzpy as xyz
# from xyzpy.gen.combo_runner import multi_concat
import multilayer_simulator as ml
from multilayer_simulator.lumerical_classes import LumericalOscillator, format_stackrt, format_stackfield
from multilayer_simulator.helpers.mixins import convert_wavelength_and_frequency
import dask
from functools import partial
from pathlib import Path
import sys
from tqdm import tqdm
# plotting
import hvplot.xarray
import hvplot.pandas
import holoviews as hv
from holoviews import dim, opts
import colorcet
import panel as pn
import panel.widgets as pnw
from bokeh.io import export_png, export_svg
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
from scipy.signal import find_peaks
from bokeh.models import PrintfTickFormatter

In [ ]:
hv.extension("bokeh", inline=False, case_sensitive_completion=True)  # use matplotlib because rendering bokeh to svg is broken
pn.config.throttled = True  # don't update interactive plots until mouse is unclicked

# default_color_cycle = hv.Cycle("Colorblind")  # Ruth doesn't like the inclusion of yellow, which is fair enough
default_color_cycle = hv.Cycle(colorcet.glasbey_dark)
default_dash_cycle = hv.Cycle(["solid", "dashed", "dashdot", "dotted", "dotdash"])
universal_opts = dict(fontscale=2, title="")
matplotlib_opts = dict(fig_inches=5, aspect=2, fig_latex=True)
bokeh_opts = dict(width=700, height=300)
opts.defaults(opts.Curve(**universal_opts|bokeh_opts, color=default_color_cycle, line_width=1.5),
              opts.Scatter(**universal_opts|bokeh_opts, color=default_color_cycle),
              opts.Image(**universal_opts|bokeh_opts),
              opts.Slope(**universal_opts|bokeh_opts, color=default_color_cycle),
              opts.Area(**universal_opts|bokeh_opts, color=default_color_cycle),
              opts.Overlay(**universal_opts|bokeh_opts),
              opts.Layout(**universal_opts|bokeh_opts),
              opts.GridSpace(**universal_opts|bokeh_opts),
              )

xarray_engine='h5netcdf'

In [ ]:
root = Path.cwd().parent.parent.parent  # depth of parents depends on if this is running in JupyterLab or Notebook

In [ ]:
code_path = root / r"research"

In [ ]:
data_path = code_path / r"notebooks/data"

In [ ]:
archive_path = root / r"thesis/LaTeX/chapters/intro"

In [ ]:
fig_path = archive_path / "fig_intro"

In [ ]:
if not code_path in sys.path:
    sys.path.append(str(code_path))
from LOPC import LOPC
from LOPC.helpers import (
    assign_derived_attrs,
    restack,
    enhancement_factor,
    # combo_length,
    # estimate_combo_run_time,
    linewidth_calculator,
    lopc_data,
    spectrum,
    normalise_over_dim,
    integrate_da,
    sel_or_integrate,
    find_optimum_coords,
    plot_secondary,
    pre_process_for_plots,
    vlines,
    coordinate_string,
    plot_da,
    plot_var,
    plot_optimum_over_dim,
    plot_field,
    visualise_multilayer,
    complex_elements,
    indexer_from_dataset,
    fix_bin_labels,
    mean_and_std,
    assign_high_from_mean_and_std,
    assign_low_from_mean_and_std,
    assign_high_and_low,
    max_min_pos,
)

Turn on auto-archiving of cells and Holoviews outputs. See the user guide [here](https://holoviews.org/user_guide/Exporting_and_Archiving.html).

Might need to install `ipympl`.

EDIT: This does not work but I'm leaving this here so a future researcher can avoid the rabbithole I fell down.

In [ ]:
# # This is the idiomatic way to record all generated figures with holoviews
# # This does NOT work in JupyterLab: see https://github.com/holoviz/holoviews/issues/3570
# # This also does not work in Jupyter Notebook
# # It's just utterly broken

# hv.archive.auto(root=str(archive_path), export_name="fig_chapter_2") 

# Plots

## Pre-processing

In [ ]:
blue = hv.Cycle.default_cycles['default_colors'][0]
red = hv.Cycle.default_cycles['default_colors'][1]
yellow = hv.Cycle.default_cycles['default_colors'][2]
green = hv.Cycle.default_cycles['default_colors'][3]

In [ ]:
period_dim = hv.Dimension("period", label="Λ", unit="nm")
wavelength_dim = hv.Dimension("wavelength", label="λ", unit="nm")
real_index_dim = hv.Dimension("n")
imag_index_dim = hv.Dimension("k", label="ϰ")
delta_dim = hv.Dimension("delta", label="Δ")
elt_dim = hv.Dimension("excitonic_layer_thickness", label="d_e", unit="nm")

In [ ]:
opts_R = [
    opts.Curve(color=blue, ylim=(0, 1)),
    opts.Image(cmap="viridis", clim=(0, 1)),
    opts.QuadMesh(cmap="viridis", clim=(0, 1)),
]


def plot_R(
    variable="R",
    dataset=None,
    label_field="long_name",
    label_append=None,
    **hvplot_kwargs
):
    plot = plot_var(variable, dataset, label_field, label_append, **hvplot_kwargs)
    plot.opts(*opts_R)
    return plot

## Bragg reflector

In [ ]:
fdtd = lumapi.FDTD()

In [ ]:
n1 = 1
n2 = 1.46
lambda_0 = 550e-9
d1 = lambda_0/(4*n1)
d2 = lambda_0/(4*n2)
num_periods = 10

In [ ]:
print(d1*1e9, d2*1e9)

In [ ]:
f = d1/(d1+d2)

In [ ]:
n_eff = np.sqrt(f*(n1**2) + (1-f)*(n2**2))

In [ ]:
n_eff

In [ ]:
2 * n_eff * (d1 + d2)

In [ ]:
lambda_0/(2*(d1+d2))

In [ ]:
wavelengths = np.linspace(400e-9, 700e-9, 1000)
frequencies = convert_wavelength_and_frequency(wavelengths)

In [ ]:
material_1 = ml.material.ConstantIndex(n1)

In [ ]:
material_2 = ml.material.ConstantIndex(n2)

In [ ]:
incident_layer = exit_layer = ml.structure.Layer.from_material(material=material_1)

In [ ]:
layer_1 = ml.structure.Layer.from_material(material=material_1, thickness=d1)

In [ ]:
layer_2 = ml.structure.Layer.from_material(material=material_2, thickness=d2)

In [ ]:
unit_cell = [layer_2, layer_1]

In [ ]:
bragg_reflector = multilayer_simulator.structure.Multilayer.from_given_unit_cell(unit_cell=unit_cell, incident_layer=incident_layer, exit_layer=exit_layer, num_periods=num_periods)

In [ ]:
engine = ml.lumerical_classes.STACKRT(session=fdtd)

In [ ]:
formatter = format_stackrt(
    output_format="xarray_dataset",
    **{
        "variables": ["Rs", "Rp", "Ts", "Tp"],
        "add_absorption": False,
    }
)

In [ ]:
simulation = ml.Simulation(structure=bragg_reflector, engine=engine, frequencies=frequencies, angles=[0, 30], formatter=formatter)

In [ ]:
data = simulation.simulate()

In [ ]:
data = assign_derived_attrs(
    data,
    absorption=True,
    unpolarised=True,
    period=False,
    total_excitonic_thickness=False,
    total_passive_thickness=False,
    total_thickness=False,
    N_tot=False,
    per_oscillator=None,
)

In [ ]:
data = pre_process_for_plots(data)

In [ ]:
the_plot = plot_R(dataset=data, x="wavelength").overlay("theta") * hv.VLine(550)
the_plot.opts(opts.Curve(ylabel="R", line_dash=hv.Cycle(["solid", "dashed"])), opts.VLine(line_dash="dotted", line_color=yellow))

In [ ]:
hv.save(the_plot, filename=fig_path / "bragg_reflector_spectrum", fmt="png", toolbar=False)